In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys

sys.path.append('./dashboard')

In [3]:
from typing import List, Union, Literal, TypedDict, Optional
import os

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.exceptions import OutputParserException
from langchain_anthropic import ChatAnthropic
from langchain_community.callbacks import get_openai_callback

In [4]:
from pydantic import (
    BaseModel,
    ValidationError,
    ValidationInfo,
    field_validator,
)
from pydantic.v1 import BaseModel as BaseModelV1

In [5]:
from dashboard.model import get_model

## OpenAI

In [6]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

## Anthropic

In [1]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    base_url=os.environ.get("ANTHROPIC_API_BASE")
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-opus-20240229",
)
print(message.content)

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

## Choose model

In [67]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# model = ChatAnthropic(
#     model='claude-3-opus-20240229',
#     anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     anthropic_api_url=os.environ.get("ANTHROPIC_API_BASE")
# )

## Trying actual Vizro models

In [9]:
import vizro.models as vm

In [10]:
vm.Page

vizro.models._page.Page

In [11]:
vm.Graph

vizro.models._components.graph.Graph

### Build planner

- Mirror vm 
- Content by hand
- include the prompt to submit
- structure of this mirroring exact structure
- recurse through the tree
- order to traverse order the fields are written in the models
#BUG: order fields matters

In [74]:
from vizro.models.types import ComponentType, ControlType

comp_type = Literal["AgGrid", "Card", "Graph"]
contr_type = Literal["Filter"]

In [75]:
class FilterPlanner(BaseModelV1):
    description: str = Field(...,
                             description="Description of the Filter. Include everything that seems to relate to this component.")
    column: Optional[str] = Field(None, description="Column to filter on.")

In [76]:
class Component(BaseModelV1):
    #    """Permissible options to describe components. Needs to be in the format of {"component_name": XXX, "component_description": XXX}."""
    component_name: comp_type
    component_description: str = Field(...,
                                       description="Description of the component. Include everything that seems to relate to this component.")


class Components(BaseModelV1):
    components: List[Component]


class Control(BaseModelV1):
    #    """Permissible options to describe controls. Needs to be in the format of {"control_name": XXX, "control_description": XXX}."""
    control_name: contr_type
    control_description: str = Field(...,
                                     description="Description of the control. Include everything that seems to relate to this control.")


class Controls(BaseModelV1):
    controls: List[Control]

In [77]:
class PagePlanner(BaseModelV1):
    title: str = Field(...,
                       description="Title of the page. If no description is provided, make a short and concise title from the components.")
    components: Components  #List[Component]#
    controls: Controls  #Optional[List[FilterPlanner]]#List[Control]#

In [69]:
query = "I need a page with a bar and a scatter that filters on the columns 'gdpPerCap` and uses a dropdown as selector. This filter should only apply to the bar chart. The bar chart should be a stacked bar chart, while the scatter chart should be colored by the column 'continent'. I also want a table that shows the data. The title of the page should be `My wonderful jolly dashboard showing a lot of data`."

In [70]:
with get_openai_callback() as cb:
    res = get_model(query, model, result_model=PagePlanner, max_retry=2)

In [71]:
res

PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description="A stacked bar chart that filters based on the 'gdpPerCap' column."), Component(component_name='Graph', component_description="A scatter chart colored by the 'continent' column."), Component(component_name='AgGrid', component_description='A table that displays the data.')]), controls=Controls(controls=[Control(control_name='Filter', control_description="A dropdown selector that applies a filter on the 'gdpPerCap' column, affecting only the bar chart.")]))

In [72]:
for i in res.components.components:
    print(i)

component_name='Graph' component_description="A stacked bar chart that filters based on the 'gdpPerCap' column."
component_name='Graph' component_description="A scatter chart colored by the 'continent' column."
component_name='AgGrid' component_description='A table that displays the data.'


In [73]:
cb

Tokens Used: 793
	Prompt Tokens: 612
	Completion Tokens: 181
Successful Requests: 1
Total Cost (USD): $0.011550000000000001

Summary:
While a lot can be teaked, it seems that the best approach is to use pydantic models that are small, Literal to extract the type of component. I am still a little confused as to why the specialised models (see in Instructor exploration) didn't help.

In this task, GPT-4 also seems a fair bit more powerful

### Going multi page

In [79]:
class DashboardPlanner(BaseModelV1):
    pages: List[PagePlanner]

In [80]:
query = "I need overall two pages. On the first page I want a bar chart and a scatter chart that filters on the columns 'gdpPerCap` and uses a dropdown as selector. This filter should only apply to the bar chart. The bar chart should be a stacked bar chart, while the scatter chart should be colored by the column 'continent'. I also want a table that shows the data. The title of the page should be `My wonderful jolly dashboard showing a lot of data`. On the second page I would like to have a line chart, and a card explaining that line chart."

In [81]:
with get_openai_callback() as cb:
    res = get_model(query, model, result_model=DashboardPlanner, max_retry=2)

In [85]:
res.pages[0]

PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description="A stacked bar chart that filters based on the columns 'gdpPerCap'."), Component(component_name='Graph', component_description="A scatter chart colored by the column 'continent'."), Component(component_name='AgGrid', component_description='A table that displays the data.')]), controls=Controls(controls=[Control(control_name='Filter', control_description="A dropdown selector that applies a filter to the bar chart based on the columns 'gdpPerCap'.")]))

In [86]:
for i in res.pages[0].components.components:
    print(i)

component_name='Graph' component_description="A stacked bar chart that filters based on the columns 'gdpPerCap'."
component_name='Graph' component_description="A scatter chart colored by the column 'continent'."
component_name='AgGrid' component_description='A table that displays the data.'


In [84]:
res.pages[1]

PagePlanner(title='Insightful Analysis with Line Chart', components=Components(components=[Component(component_name='Graph', component_description='A line chart that presents time series data.'), Component(component_name='Card', component_description='A descriptive card that explains the data and trends shown in the line chart.')]), controls=Controls(controls=[]))

### Trying in conjunction with actual data and objects

In [92]:
import plotly.express as px

df = px.data.gapminder()

In [114]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# model = ChatAnthropic(
#     model='claude-3-opus-20240229',
#     anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     anthropic_api_url=os.environ.get("ANTHROPIC_BASE_URL")
# )

In [126]:
query = "I need a page with a bar chart and a scatter chart that filters on the GDP column and uses a dropdown as selector. This filter should only apply to the bar chart. The bar chart should be a stacked bar chart, while the scatter chart should be colored by the column 'continent'. I also want a table that shows the data. The title of the page should be `My wonderful jolly dashboard showing a lot of data`."

In [127]:
with get_openai_callback() as cb:
    res = get_model(query, model, result_model=PagePlanner, max_retry=2)

In [128]:
res.controls.controls[0]

Control(control_name='Filter', control_description='Dropdown selector to filter GDP column for the bar chart')

In [129]:
model_instruction = res.controls.controls[0].control_description
selected_query = f"Create a filter from the following instructions: {model_instruction}. Do not make up things that are optional. Consider the context of the data {df.head()}. If no options are specified, leave them out."

In [130]:
selected_query

'Create a filter from the following instructions: Dropdown selector to filter GDP column for the bar chart. Do not make up things that are optional. Consider the context of the data        country continent  year  lifeExp       pop   gdpPercap iso_alpha  \\\n0  Afghanistan      Asia  1952   28.801   8425333  779.445314       AFG   \n1  Afghanistan      Asia  1957   30.332   9240934  820.853030       AFG   \n2  Afghanistan      Asia  1962   31.997  10267083  853.100710       AFG   \n3  Afghanistan      Asia  1967   34.020  11537966  836.197138       AFG   \n4  Afghanistan      Asia  1972   36.088  13079460  739.981106       AFG   \n\n   iso_num  \n0        4  \n1        4  \n2        4  \n3        4  \n4        4  . If no options are specified, leave them out.'

In [131]:
with get_openai_callback() as cb:
    res2 = get_model(selected_query, model, result_model=vm.Filter, max_retry=2)

In [132]:
res2

Filter(id='eb2083e6-ce16-4dba-0ff1-8e0242af9fc3', type='filter', column='gdpPercap', targets=[], selector=Dropdown(id='17e0aa3c-0398-3ca8-ea7e-9d498c778ea6', type='dropdown', options=[], value=None, multi=True, title='', actions=[]))

In [133]:
vm.Page

vizro.models._page.Page

## Real deal

In [139]:
component_type = Literal["AgGrid", "Card", "Graph"]
control_type = Literal["Filter"]


class Component(BaseModelV1):
    component_name: component_type
    component_description: str = Field(...,
                                       description="Description of the component. Include everything that seems to relate to this component.")


class Components(BaseModelV1):
    components: List[Component]


class Control(BaseModelV1):
    control_name: control_type
    control_description: str = Field(...,
                                     description="Description of the control. Include everything that seems to relate to this control.")


class Controls(BaseModelV1):
    controls: List[Control]


class PagePlanner(BaseModelV1):
    title: str = Field(...,
                       description="Title of the page. If no description is provided, make a short and concise title from the components.")
    components: Components  #List[Component]#
    controls: Controls  #Optional[List[FilterPlanner]]#List[Control]#


class DashboardPlanner(BaseModelV1):
    pages: List[PagePlanner]

In [142]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# model = ChatAnthropic(
#     model='claude-3-opus-20240229',
#     anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     anthropic_api_url=os.environ.get("ANTHROPIC_BASE_URL")
# )

In [143]:
query = "I need a page with a bar chart and a scatter chart that filters on the GDP column and uses a dropdown as selector. This filter should only apply to the bar chart. The bar chart should be a stacked bar chart, while the scatter chart should be colored by the column 'continent'. I also want a table that shows the data. The title of the page should be `My wonderful jolly dashboard showing a lot of data`."

In [144]:
with get_openai_callback() as cb:
    res = get_model(query, model, result_model=DashboardPlanner, max_retry=2)

In [145]:
res

DashboardPlanner(pages=[PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description='A stacked bar chart that visualizes data, filtered by GDP using a dropdown selector.'), Component(component_name='Graph', component_description="A scatter chart colored by the 'continent' column, showing the distribution of data points across different continents."), Component(component_name='AgGrid', component_description='A table that displays the data in a structured format, allowing for easy viewing and analysis.')]), controls=Controls(controls=[Control(control_name='Filter', control_description='A dropdown selector used to filter the bar chart based on the GDP column.')]))])

In [148]:
def get_dashboard_plan(query: str, model: Union[ChatOpenAI, ChatAnthropic],
                       max_retry: int = 3) -> PagePlanner:
    return get_model(query, model, result_model=DashboardPlanner, max_retry=max_retry)

In [149]:
get_dashboard_plan(query, model)

DashboardPlanner(pages=[PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description='A stacked bar chart that visualizes data, filtered by GDP using a dropdown selector.'), Component(component_name='Graph', component_description="A scatter chart colored by the 'continent' column, showing the relationship between different data points."), Component(component_name='AgGrid', component_description='A table that displays the underlying data for the charts.')]), controls=Controls(controls=[Control(control_name='Filter', control_description='A dropdown selector used to filter data in the bar chart based on the GDP column.')]))])